# Build LSTM model

## Read data

In [1]:
import os
from pathlib import Path

import pandas as pd


dataDirName = Path(os.environ['DATA_DIR'], 'sepsis_prediction', 'lstm_initial_trials', '02_data_matrix', 'balanced_data')
dataDirName.mkdir(exist_ok=True, parents=True)

dataDf = pd.read_csv(Path(dataDirName, 'data_temp_variables.csv'))
dataDf = dataDf.sort_values(by=['PATIENT_ID', 'EPISODE_ID', 'measurement_datetime'], ascending=True)
dataDf

,PATIENT_ID,EPISODE_ID,measurement_datetime,concept_name,value_mean,value_std,value_open,value_high,value_low,value_close,Organism_FIRST_NOTED,sepsis,target
0,56749,14126754,2020-11-23,Temperature,36.619017,0.000000,0.0,0.0,0.0,0.0,2021-07-30,0.0,0.0
1,56749,14126754,2020-11-24,Temperature,37.150000,0.288675,37.4,37.4,36.9,36.9,2021-07-30,0.0,0.0
2,56749,14126754,2020-11-25,Temperature,36.800000,0.000000,36.8,36.8,36.8,36.8,2021-07-30,0.0,0.0
3,62567,12648702,2019-07-25,Temperature,37.771429,0.596418,38.6,38.6,36.9,37.7,2019-07-26,1.0,0.0
4,62567,12648702,2019-07-26,Temperature,37.000000,0.427618,37.2,37.8,36.3,36.8,2019-07-26,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27370,2669870,17327467,2023-02-03,Temperature,36.300000,0.327872,36.1,37.0,36.0,37.0,2023-01-23,0.0,0.0
27371,2669870,17327467,2023-02-04,Temperature,36.750000,0.361939,37.2,37.2,36.1,36.8,2023-01-23,0.0,0.0
27372,2669870,17327467,2023-02-05,Temperature,36.483333,0.292689,36.2,36.8,36.1,36.8,2023-01-23,0.0,0.0
27373,2669870,17327467,2023-02-06,Temperature,36.583333,0.392003,36.8,37.0,36.0,36.7,2023-01-23,0.0,0.0


In [2]:
rowsX = []
rowsY = []
for name, group in dataDf.groupby(by=['PATIENT_ID', 'EPISODE_ID']):
    low = 0
    high = 3
    while high <= group.shape[0]:
        rowsX.append([[x] for x in list(group.value_mean[low: high])])
        rowsY.append(group.target[(high - 1): high].values[0])
        high += 1
        low += 1

In [3]:
rowsX

[[[36.61901735331418], [37.15], [36.8]],
 [[37.771428571428565], [37.0], [36.66666666666666]],
 [[37.0], [36.66666666666666], [37.016666666666666]],
 [[36.66666666666666], [37.016666666666666], [36.81666666666667]],
 [[37.016666666666666], [36.81666666666667], [36.75]],
 [[36.81666666666667], [36.75], [36.72]],
 [[36.75], [36.72], [36.9]],
 [[36.72], [36.9], [36.61901735331418]],
 [[36.9], [36.61901735331418], [36.0]],
 [[36.61901735331418], [36.0], [36.6]],
 [[36.0], [36.6], [36.9]],
 [[36.6], [36.9], [36.5]],
 [[36.9], [36.5], [36.61901735331418]],
 [[36.5], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.61901735331418]],
 [[36.61901735331418], [36.61901735331418], [36.466666666666

In [4]:
import numpy as np


trainX = np.array(rowsX)
testX = np.array(rowsX)
trainY = np.array(rowsY)
testY = np.array(rowsY)


In [5]:
trainX.shape, trainY.shape

((24733, 3, 1), (24733,))

### LSTM

In [6]:
# LSTM for international airline passengers problem with time step regression framing
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


look_back = 3

model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

2025-05-13 04:29:20.801696: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 04:29:20.802696: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 04:29:20.805717: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 04:29:20.813995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747110560.828041  355466 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747110560.83

Epoch 1/10


2025-05-13 04:29:22.468047: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24733/24733 - 25s - 994us/step - accuracy: 0.5823 - loss: 0.6802
Epoch 2/10
24733/24733 - 25s - 1ms/step - accuracy: 0.5827 - loss: 0.6797
Epoch 3/10
24733/24733 - 24s - 980us/step - accuracy: 0.5827 - loss: 0.6798
Epoch 4/10
24733/24733 - 24s - 963us/step - accuracy: 0.5827 - loss: 0.6796
Epoch 5/10
24733/24733 - 23s - 944us/step - accuracy: 0.5827 - loss: 0.6797
Epoch 6/10
24733/24733 - 23s - 935us/step - accuracy: 0.5827 - loss: 0.6796
Epoch 7/10
24733/24733 - 23s - 937us/step - accuracy: 0.5827 - loss: 0.6796
Epoch 8/10
24733/24733 - 23s - 936us/step - accuracy: 0.5827 - loss: 0.6798
Epoch 9/10
24733/24733 - 23s - 942us/step - accuracy: 0.5827 - loss: 0.6796
Epoch 10/10
24733/24733 - 23s - 945us/step - accuracy: 0.5827 - loss: 0.6797
773/773 ━━━━━━━━━━━━━━━━━━━━ 1s 651us/step
773/773 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


print('accuracy_score', accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('balanced_accuracy_score', balanced_accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_score', precision_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('recall_score', recall_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_recall_fscore_support', precision_recall_fscore_support(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('f1_score', f1_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('roc_auc_score', roc_auc_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))

accuracy_score 0.582662839121821
balanced_accuracy_score 0.5
precision_score 0.0
recall_score 0.0
precision_recall_fscore_support (array([0.58266284, 0.        ]), array([1., 0.]), array([0.73630697, 0.        ]), array([14411, 10322]))
f1_score 0.0
roc_auc_score 0.5


/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
print('roc_auc_score', roc_auc_score(trainY, trainPredict))

roc_auc_score 0.4999653042814517


## DNN

In [9]:
from tensorflow import keras


model = keras.Sequential(
    [
        keras.Input(shape=(3, )),
        keras.layers.Dense(4, activation="relu"),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 4)              │            16 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21 (84.00 B)

 Trainable params: 21 (84.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.Accuracy(name="accuracy"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

model.fit(trainX.reshape((trainX.shape[0], 3)), trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
24733/24733 - 22s - 874us/step - accuracy: 3.2345e-04 - fn: 10261.0000 - fp: 85.0000 - loss: 0.7015 - precision: 0.4178 - recall: 0.0059 - tn: 14326.0000 - tp: 61.0000
Epoch 2/10
24733/24733 - 21s - 859us/step - accuracy: 0.0000e+00 - fn: 10322.0000 - fp: 0.0000e+00 - loss: 0.6808 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 14411.0000 - tp: 0.0000e+00
Epoch 3/10
24733/24733 - 21s - 850us/step - accuracy: 0.0000e+00 - fn: 10322.0000 - fp: 0.0000e+00 - loss: 0.6808 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 14411.0000 - tp: 0.0000e+00
Epoch 4/10
24733/24733 - 20s - 823us/step - accuracy: 0.0000e+00 - fn: 10322.0000 - fp: 0.0000e+00 - loss: 0.6809 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 14411.0000 - tp: 0.0000e+00
Epoch 5/10
24733/24733 - 20s - 828us/step - accuracy: 0.0000e+00 - fn: 10322.0000 - fp: 0.0000e+00 - loss: 0.6804 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 14411.0000 - tp: 0.0000e+00
Epoch 6/10
24733/24733 - 20s - 828us/step - accur

In [11]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

  1/773 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step

773/773 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step
773/773 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step


In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


print('accuracy_score', accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('balanced_accuracy_score', balanced_accuracy_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_score', precision_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('recall_score', recall_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('precision_recall_fscore_support', precision_recall_fscore_support(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('f1_score', f1_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))
print('roc_auc_score', roc_auc_score(trainY, [1 if (x[0] > 0.5) else 0 for x in trainPredict]))

accuracy_score 0.582662839121821
balanced_accuracy_score 0.5


precision_score 0.0
recall_score 0.0
precision_recall_fscore_support (array([0.58266284, 0.        ]), array([1., 0.]), array([0.73630697, 0.        ]), array([14411, 10322]))
f1_score 0.0
roc_auc_score 0.5


/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
print('roc_auc_score', roc_auc_score(trainY, trainPredict))

roc_auc_score 0.5


## Traditional ML

### NB Classifier model

In [14]:
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()
y_score = gnb.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = gnb.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict(trainX.reshape((trainX.shape[0], 3)))


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.582662839121821
Balanced Accuracy 0.5
F1 Score 0.0
Precision Score 0.0
Recall Score 0.0
roc_auc_score 0.5301128349674651


/home/vmadmin/workspace/sepsis_predictor/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### LR Classifier

In [16]:
from sklearn.linear_model import LogisticRegression


lrc = LogisticRegression(random_state=0)
y_score = lrc.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = lrc.fit(trainX.reshape((trainX.shape[0], 3)), trainY).predict(trainX.reshape((trainX.shape[0], 3)))

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.5820967937573283
Balanced Accuracy 0.49983038358325255
F1 Score 0.0044307455210942015
Precision Score 0.38333333333333336
Recall Score 0.0022282503390815733
roc_auc_score 0.5323054013549764


### XGBoost Classifier

In [18]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(trainX.reshape((trainX.shape[0], 3)), trainY)
# make predictions
y_score = bst.predict_proba(trainX.reshape((trainX.shape[0], 3)))
y_pred = bst.predict(trainX.reshape((trainX.shape[0], 3)))

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print('Accuracy', accuracy_score(trainY, y_pred))
print('Balanced Accuracy', balanced_accuracy_score(trainY, y_pred))
print('F1 Score', f1_score(trainY, y_pred))
print('Precision Score', precision_score(trainY, y_pred))
print('Recall Score', recall_score(trainY, y_pred))
print('roc_auc_score', roc_auc_score(trainY, [y[1] for y in y_score]))

Accuracy 0.5847652933327943
Balanced Accuracy 0.5055151940423774
F1 Score 0.04995374653098982
Precision Score 0.5532786885245902
Recall Score 0.026157721371827165
roc_auc_score 0.5458594172509532
